**Extracting diagnostic information on species growth!**
Method: 
1) TRAIN the model on all data from the census.
2) For each species present, create a dataset where all the trees are of that one species and every DBH from 10 to 500 is represented.
3) Use this dataset as a test set.
4) Instead of calculating a loss function, plot the output as a function of the DBH input!

In [12]:
import numpy as np
import scipy as sp
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
import category_encoders as ce
import xgboost as xgb
import matplotlib.pyplot as mpl
import data_processing

In [13]:
# attempt to use data processing script!
paths = ("/Users/dylanvanbramer/indresearch/xu/deep_learning/bci_census/bci_1985.csv","/Users/dylanvanbramer/indresearch/xu/deep_learning/bci_census/bci_1990.csv")
feats, labels, test_ids, test_feats = data_processing.clean(paths)


TypeError: _load_data() missing 1 required positional argument: 'quads'

In [ ]:
tree = RandomForestRegressor()
tree.fit(feats, labels)

tree2 = xgb.XGBRegressor()
tree2.fit(feats,labels)

In [ ]:
# Proof of concept for first species!

range = np.arange(10,501).transpose()
sp_label = np.atleast_2d(quad1_t1[0,1:8])
sp_repeated = np.repeat(sp_label,491,axis=0)
print (sp_repeated)
sp_1 = np.column_stack((sp_repeated,range))

predictions_sp1 = tree.predict(sp_1)
mpl.plot(range, predictions_sp1)

In [ ]:
# first challenge: how to know which species is which - need to keep a list before encoding perhaps?
list_species = species_list = []
    for i in quad1_t1.index:
        current = quad1_t1['sp'][i] 
        print (current)
        if current not in species_list:
                species_list.append(current)
        quad1_t1['sp'][i]  = species_list.index(current)

    print (len(species_list))
    quad1_t1.head(30)
    print(species_list)
    quad1_t1.head()



encoder= ce.BinaryEncoder(cols=['sp'],return_df=True)

data_encoded=encoder.fit_transform(quad1_t1)

data_encoded.head()

# arange from 10 to 500

# hstack